# Use prebuilt Azure AI Translator in Fabric with REST API and SynapseML

### Set up authentication and endpoints

In [1]:
# Get workload endpoints and access token

from synapse.ml.fabric.service_discovery import get_fabric_env_config
from synapse.ml.fabric.token_utils import TokenUtils
import json
import requests


fabric_env_config = get_fabric_env_config().fabric_env_config
auth_header = TokenUtils().get_openai_auth_header()

# Make a RESTful request to AI service
prebuilt_AI_base_host = fabric_env_config.ml_workload_endpoint + "cognitive/texttranslation/"
print("Workload endpoint for AI service: \n" + prebuilt_AI_base_host)

service_url = prebuilt_AI_base_host + "language/:analyze-text?api-version=2022-05-01"
print("Service URL: \n" + service_url)

auth_headers = {
    "Authorization" : auth_header
}

def print_response(response):
    print(f"HTTP {response.status_code}")
    if response.status_code == 200:
        try:
            result = response.json()
            print(json.dumps(result, indent=2, ensure_ascii=False))
        except:
            print(f"pasre error {response.content}")
    else:
        print(f"error message: {response.content}")

StatementMeta(, fbd5a888-5396-47c6-8d33-da9f63155f9c, 3, Finished, Available, Finished)

Workload endpoint for AI service: 
https://pbipindcen2-centralindia.pbidedicated.windows.net/webapi/capacities/d40e324e-1d9e-4953-aed7-9f4787b9536b/workloads/ML/ML/Automatic/workspaceid/813bdb96-27f1-4c2a-b79c-c24a08722010/cognitive/texttranslation/
Service URL: 
https://pbipindcen2-centralindia.pbidedicated.windows.net/webapi/capacities/d40e324e-1d9e-4953-aed7-9f4787b9536b/workloads/ML/ML/Automatic/workspaceid/813bdb96-27f1-4c2a-b79c-c24a08722010/cognitive/texttranslation/language/:analyze-text?api-version=2022-05-01


### Text Translation
Text translation is the core operation of the Translator service.

In [2]:
service_url = prebuilt_AI_base_host + "translate?api-version=3.0&to=fr"
post_body = [{'Text':'Hello, friend.'}]

response = requests.post(service_url, json=post_body, headers=auth_headers)

# Output all information of the request process
print_response(response)

StatementMeta(, fbd5a888-5396-47c6-8d33-da9f63155f9c, 4, Finished, Available, Finished)

HTTP 200
[
  {
    "detectedLanguage": {
      "language": "en",
      "score": 1.0
    },
    "translations": [
      {
        "text": "Bonjour cher ami.",
        "to": "fr"
      }
    ]
  }
]


### Text Transliteration
Transliteration converts a word or phrase from the script (alphabet) of one language to another, based on phonetic similarity.

In [3]:
service_url = prebuilt_AI_base_host + "transliterate?api-version=3.0&language=ja&fromScript=Jpan&toScript=Latn"
post_body = [
    {"Text":"こんにちは"},
    {"Text":"さようなら"}
]

response = requests.post(service_url, json=post_body, headers=auth_headers)

# Output all information of the request process
print_response(response)

StatementMeta(, fbd5a888-5396-47c6-8d33-da9f63155f9c, 5, Finished, Available, Finished)

HTTP 200
[
  {
    "text": "Kon'nichiwa​",
    "script": "Latn"
  },
  {
    "text": "Sayou nara",
    "script": "Latn"
  }
]


### Supported Languages Retrieval
Returns a list of languages that Translator operations support.

In [5]:
service_url = prebuilt_AI_base_host + "languages?api-version=3.0"

response = requests.get(service_url, headers=auth_headers)

# Output all information of the request process
# print_response(response)

StatementMeta(, fbd5a888-5396-47c6-8d33-da9f63155f9c, 7, Finished, Available, Finished)

In [9]:
# Convert to DataFrame
data = response.json()
rows = [
    {"code": code, **info}
    for code, info in data["translation"].items()
]
df = spark.createDataFrame(rows)
display(df)

StatementMeta(, fbd5a888-5396-47c6-8d33-da9f63155f9c, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 52284d98-0920-4bc2-a67a-883caed2534f)

## SynapseML approach

#### Import required libraries

In [10]:
import synapse.ml.core
from synapse.ml.services import *
from pyspark.sql.functions import col, flatten

StatementMeta(, fbd5a888-5396-47c6-8d33-da9f63155f9c, 12, Finished, Available, Finished)

### Text Translation

In [13]:
df = spark.createDataFrame([
  (["Hello, what is your name?", "Bye"],)
], ["text",])

translate = (Translate()
    .setTextCol("text")
    .setToLanguage(["zh-Hans", "fr"])
    .setOutputCol("translation")
    .setConcurrency(5))

result = translate.transform(df)\
        .withColumn("translation", flatten(col("translation.translations")))\
        .withColumn("translation", col("translation.text"))

display(result.select("text", "translation"))

StatementMeta(, fbd5a888-5396-47c6-8d33-da9f63155f9c, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, da6283bc-a50e-4c9c-9b75-ec0493597691)

### Text Transliteration

In [14]:
transliterateDf =  spark.createDataFrame([
  (["こんにちは", "さようなら"],)
], ["text",])

transliterate = (Transliterate()
    .setLanguage("ja")
    .setFromScript("Jpan")
    .setToScript("Latn")
    .setTextCol("text")
    .setOutputCol("result"))

result = transliterate.transform(transliterateDf)\
        .withColumn("script", col("result.script"))

display(result.select("text", "script"))

StatementMeta(, fbd5a888-5396-47c6-8d33-da9f63155f9c, 16, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 051ca77a-8023-4142-a4ca-24c8d03f8e7f)